In [209]:
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import joblib

In [157]:
url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EB%82%A0%EC%94%A8'
url1 = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EB%AF%B8%EC%84%B8%EB%A8%BC%EC%A7%80'
url2 = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EC%B4%88%EB%AF%B8%EC%84%B8%EB%A8%BC%EC%A7%80'

In [158]:
driver = webdriver.Chrome()
driver.get(url)
time.sleep(1)

In [159]:
time.sleep(1)
driver.find_element(By.CLASS_NAME, 'title_select').click()

In [160]:
driver.find_elements(By.CLASS_NAME, 'item_link')[1].click()
time.sleep(1)

In [161]:
time.sleep(1)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [162]:
temperatures = [int(soup.select('.temperature_text')[0].text.strip().split('도')[1].strip('°'))]

In [163]:
for i in range(23):
    temperatures.append(int(soup.select('.num')[i].text.strip('°')))

In [164]:
temperature = sum(temperatures) / len(temperatures)

In [165]:
temperature

15.083333333333334

In [166]:
rains = []
for i in range(24):
    rains.append(float(soup.select('.data_inner')[i].text.strip()))

In [167]:
rain = sum(rains)

In [168]:
rain

0.0

In [169]:
humidities = [int(soup.select('.sort')[1].text.split()[1].strip('%'))]

In [170]:
for i in range(23):
    humidities.append(int(soup.select('.num')[i+144].text))

In [171]:
humidity = sum(humidities) / len(humidities)

In [172]:
humidity

40.583333333333336

In [188]:
def convert_wind(wind):
    way = wind.split()[0]
    speed = float(wind.split()[1])
    converted_wind = np.zeros(16)
    if way == '북풍':
        converted_wind = np.array([speed, 0, 0, 0, 0, 0, 0, 0,
                                   0, 0, 0, 0, 0, 0, 0, 0])
    if way == '북북동풍':
        converted_wind = np.array([0, speed, 0, 0, 0, 0, 0, 0,
                                   0, 0, 0, 0, 0, 0, 0, 0])
    if way == '북동풍':
        converted_wind = np.array([0, 0, speed, 0, 0, 0, 0, 0,
                                   0, 0, 0, 0, 0, 0, 0, 0])
    if way == '동북동풍':
        converted_wind = np.array([0, 0, 0, speed, 0, 0, 0, 0,
                                   0, 0, 0, 0, 0, 0, 0, 0])
    if way == '동풍':
        converted_wind = np.array([0, 0, 0, 0, speed, 0, 0, 0,
                                   0, 0, 0, 0, 0, 0, 0, 0])
    if way == '동남동풍':
        converted_wind = np.array([0, 0, 0, 0, 0, speed, 0, 0,
                                   0, 0, 0, 0, 0, 0, 0, 0])
    if way == '남동풍':
        converted_wind = np.array([0, 0, 0, 0, 0, 0, speed, 0,
                                   0, 0, 0, 0, 0, 0, 0, 0])
    if way == '남남동':
        converted_wind = np.array([0, 0, 0, 0, 0, 0, 0, speed,
                                   0, 0, 0, 0, 0, 0, 0, 0])
    if way == '남풍':
        converted_wind = np.array([0, 0, 0, 0, 0, 0, 0, 0,
                                   speed, 0, 0, 0, 0, 0, 0, 0])
    if way == '남남서':
        converted_wind = np.array([0, 0, 0, 0, 0, 0, 0, 0,
                                   0, speed, 0, 0, 0, 0, 0, 0])
    if way == '남서풍':
        converted_wind = np.array([0, 0, 0, 0, 0, 0, 0, 0,
                                   0, 0, speed, 0, 0, 0, 0, 0])
    if way == '서남서풍':
        converted_wind = np.array([0, 0, 0, 0, 0, 0, 0, 0,
                                   0, 0, 0, speed, 0, 0, 0, 0])
    if way == '서풍':
        converted_wind = np.array([0, 0, 0, 0, 0, 0, 0, 0,
                                   0, 0, 0, 0, speed, 0, 0, 0])
    if way == '서북서풍':
        converted_wind = np.array([0, 0, 0, 0, 0, 0, 0, 0,
                                   0, 0, 0, 0, 0, speed, 0, 0])
    if way == '북서풍':
        converted_wind = np.array([0, 0, 0, 0, 0, 0, 0, 0,
                                   0, 0, 0, 0, 0, 0, speed, 0])
    if way == '북북서풍':
        converted_wind = np.array([0, 0, 0, 0, 0, 0, 0, 0,
                                   0, 0, 0, 0, 0, 0, 0, speed])
    return converted_wind

In [189]:
winds = [soup.select('.sort')[2].text.replace('\n',' ').strip().strip('m/s')]

In [190]:
for i in range(23):
    winds.append(soup.select('.value')[i+72].text + ' ' + soup.select('.num')[i+72].text)

In [191]:
winds

['서북서풍 2.7',
 '서북서풍 5',
 '서북서풍 5',
 '서풍 5',
 '서남서풍 4',
 '서남서풍 3',
 '서남서풍 3',
 '서남서풍 2',
 '서남서풍 2',
 '서남서풍 2',
 '남서풍 2',
 '남풍 2',
 '남동풍 2',
 '동남동풍 2',
 '동남동풍 2',
 '동남동풍 2',
 '동풍 2',
 '동풍 2',
 '동풍 2',
 '동풍 3',
 '동남동풍 3',
 '남풍 3',
 '남남서풍 3',
 '남서풍 3']

In [192]:
converted_winds = []
for wind in winds:
    converted_winds.append(convert_wind(wind))

In [193]:
converted_winds

[array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        2.7, 0. , 0. ]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0.

In [195]:
wind_sum = np.zeros(16)
for converted_wind in converted_winds:
    wind_sum += converted_wind
wind_sum

array([ 0. ,  0. ,  0. ,  0. ,  9. ,  9. ,  2. ,  0. ,  5. ,  0. ,  5. ,
       16. ,  5. , 12.7,  0. ,  0. ])

In [196]:
wind = wind_sum / 24
wind

array([0.        , 0.        , 0.        , 0.        , 0.375     ,
       0.375     , 0.08333333, 0.        , 0.20833333, 0.        ,
       0.20833333, 0.66666667, 0.20833333, 0.52916667, 0.        ,
       0.        ])

In [197]:
driver.close()

In [198]:
res1 = requests.get(url1)
soup1 = BeautifulSoup(res1.text, 'html.parser')

In [204]:
PM10 = int(soup1.select('td > span')[0].text)
PM10

34

In [205]:
res2 = requests.get(url2)
soup2 = BeautifulSoup(res2.text, 'html.parser')

In [206]:
PM25 = int(soup2.select('td > span')[0].text)
PM25

11

In [215]:
precipitation = [0, 0]
if rain > 0:
    precipitation = [0, 1]
else:
    precipitation = [1, 0]

In [216]:
test_set = [[PM10, PM25, temperature, rain, humidity] + list(wind) + precipitation]
test_set

[[34,
  11,
  15.083333333333334,
  0.0,
  40.583333333333336,
  0.0,
  0.0,
  0.0,
  0.0,
  0.375,
  0.375,
  0.08333333333333333,
  0.0,
  0.20833333333333334,
  0.0,
  0.20833333333333334,
  0.6666666666666666,
  0.20833333333333334,
  0.5291666666666667,
  0.0,
  0.0,
  1,
  0]]

In [217]:
scaler = joblib.load('C:/PM_predict/scaler.sav')
model = joblib.load('C:/PM_predict/final.model')

In [224]:
tst = scaler.transform(test_set)
predict = model.predict(tst)
probablity = model.predict_proba(tst)
print(predict)
print(probablity[0][1])

[0]
0.219


c:\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
